# Resolución del problema de asignación de alumnos de un curso original a uno nuevo

En esta aplicación se pretende asignar a un alumno de un curso original a uno nuevo, sujeto a distintas condiciones:

* El alumno debe estar en un solo curso.
* Los cursos resultantes deben tener una cantidad similar de alumnos.
* Cada curso resultante debe tener una proporción similar de niñas y niños.
* Cada curso resultante debe tener una proporción similar de alumnos de los cursos originales.
* Los alumnos incompatibles deben quedar en cursos distintos.
* Cada alumno debe quedar con por lo menos un compañero que haya escogido.

La implementación en Python se realizó utilizando la librería PuLP siguiendo las instrucciones de este [link](https://towardsdatascience.com/linear-programming-using-python-priyansh-22b5ee888fe0).

Para fines de mostrar cómo funciona el algoritmo, el primer paso consiste en generar una tabla con los alumnos y sus características indicadas anteriormente.

Luego se generan las variables del modelo, la función objetivo y las restricciones, ejecutando posteriormente el modelo, del cual importa menos el resultado de la optimización que la asignación de los alumnos a los distintos cursos debido a las restricciones.

Por último se genera una tabla con el resultado de la asignación y se evalúa el cumplimento de las restricciones. Nótese que es posible que no se encuentre un resultado que cumpla todas las condiciones, en cuyo caso, el algoritmo entrega un resultado preliminar para que se ajuste posteriormente.


In [86]:
pip install pulp

In [87]:
###################################################################################################################
# Problema asignacion alumnos
##################################################################################################

import pandas as pd
import numpy as np
import random
from pulp import *

pd.options.mode.chained_assignment = None 

print('setup ok')

setup ok


In [88]:
# Generacion de Datos de entrada

tot_alum=123
num_cursos_orig=4
num_cursos=3
num_escogidos=3

tam_cursos=int((tot_alum//num_cursos))

num_par_imcomp=int(tot_alum*.05) # Cantidad de pares de incompatibles

id=list(range(1,tot_alum+1))

#print(tam_cursos)
#print(id)

#Se simula el curso original del que provenia el estudiante
#curso_orig=np.random.randint(low=1, high=num_cursos_orig+1, size=tot_alum, dtype=int)
curso_orig=[]
for _ in range(1,num_cursos_orig):
  curso_orig=curso_orig+[_]*int(round((tot_alum/num_cursos_orig)))
curso_orig=curso_orig+[num_cursos_orig]*(tot_alum-len(curso_orig))
#print(curso_orig)

max_mismo_curs_orig=int((tot_alum)//(num_cursos*num_cursos_orig))
#print(max_mismo_curs_orig)

ninas=[]
for _ in range(tot_alum):
  ninas=ninas+[random.randint(0,1)]

#print(ninas)
#print(len(ninas))

ninas_por_curso=int(sum(ninas)//num_cursos)

#print(ninas_por_curso)

# list de incompatibles (no puede haber mas incompatibles que cursos a asignar)


incomp=random.sample(id, k=num_par_imcomp*2)
incompat_tp=list(zip(incomp[0:num_par_imcomp], incomp[num_par_imcomp:]))

incompatibles=np.array(incompat_tp)
incompatibles=pd.DataFrame(incompatibles, columns=['inc_1', 'inc_2'])
inc2=incompatibles[['inc_1', 'inc_2']]
inc2.columns=['inc_2', 'inc_1']
incompatibles=incompatibles.append(inc2, ignore_index=True)
incompatibles.columns=['id', 'incompatible']
incompatibles['id']=incompatibles['id'].astype('int64')
incompatibles['incompatible']=incompatibles['incompatible'].astype('int64')

n_incompat =num_par_imcomp

# Companeros escogidos 
escogidos_orig=[]
for _ in id:

  escogidos_orig.append(random.sample([x for x in id if x != _], k=num_escogidos))


escogidos=np.array(escogidos_orig)
escogidos=escogidos.T

datos_entrada=pd.DataFrame()
datos_entrada['id'] =id
datos_entrada['curso_orig'] =curso_orig
datos_entrada['nina'] =ninas

i=1
for _ in escogidos:
  datos_entrada['escogido_'+str(i)]=_
  i=i+1


datos_entrada=datos_entrada.merge(incompatibles, on='id', how='left')

#print(datos_entrada['curso_orig'].value_counts())

datos_entrada.head()



,id,curso_orig,nina,escogido_1,escogido_2,escogido_3,incompatible
0,1,1,0,22,91,111,NaN
1,2,1,1,17,83,84,NaN
2,3,1,1,108,109,12,NaN
3,4,1,1,107,18,103,NaN
4,5,1,0,75,6,26,NaN


In [89]:
# Iniciacion del modelo

model=LpProblem('Asign_alum', LpMinimize)

# Definicion de las variables de decision

variable_names = ['cur_'+str(i)+'_alum_'+str(j) for j in range(1, tot_alum+1) for i in range(1,num_cursos+1)]



DV_variables = LpVariable.matrix("X", variable_names, cat = "Integer", lowBound= 0 )

rest_1=np.array(DV_variables[0:tot_alum*num_cursos]).reshape(tot_alum,num_cursos)

print(DV_variables)
#print(rest_1)
print(len(variable_names))

[X_cur_1_alum_1, X_cur_2_alum_1, X_cur_3_alum_1, X_cur_1_alum_2, X_cur_2_alum_2, X_cur_3_alum_2, X_cur_1_alum_3, X_cur_2_alum_3, X_cur_3_alum_3, X_cur_1_alum_4, X_cur_2_alum_4, X_cur_3_alum_4, X_cur_1_alum_5, X_cur_2_alum_5, X_cur_3_alum_5, X_cur_1_alum_6, X_cur_2_alum_6, X_cur_3_alum_6, X_cur_1_alum_7, X_cur_2_alum_7, X_cur_3_alum_7, X_cur_1_alum_8, X_cur_2_alum_8, X_cur_3_alum_8, X_cur_1_alum_9, X_cur_2_alum_9, X_cur_3_alum_9, X_cur_1_alum_10, X_cur_2_alum_10, X_cur_3_alum_10, X_cur_1_alum_11, X_cur_2_alum_11, X_cur_3_alum_11, X_cur_1_alum_12, X_cur_2_alum_12, X_cur_3_alum_12, X_cur_1_alum_13, X_cur_2_alum_13, X_cur_3_alum_13, X_cur_1_alum_14, X_cur_2_alum_14, X_cur_3_alum_14, X_cur_1_alum_15, X_cur_2_alum_15, X_cur_3_alum_15, X_cur_1_alum_16, X_cur_2_alum_16, X_cur_3_alum_16, X_cur_1_alum_17, X_cur_2_alum_17, X_cur_3_alum_17, X_cur_1_alum_18, X_cur_2_alum_18, X_cur_3_alum_18, X_cur_1_alum_19, X_cur_2_alum_19, X_cur_3_alum_19, X_cur_1_alum_20, X_cur_2_alum_20, X_cur_3_alum_20, X_cur_

In [ ]:

# Funcion Objetivo

'''
Se genera el vector funcion objetivo con el numero de elementos como la combinacion
de alumnos y cursos 
'''

obj_func = lpSum(DV_variables)
print(obj_func)
model +=  obj_func
#print(model)



X_cur_1_alum_1 + X_cur_1_alum_10 + X_cur_1_alum_100 + X_cur_1_alum_101 + X_cur_1_alum_102 + X_cur_1_alum_103 + X_cur_1_alum_104 + X_cur_1_alum_105 + X_cur_1_alum_106 + X_cur_1_alum_107 + X_cur_1_alum_108 + X_cur_1_alum_109 + X_cur_1_alum_11 + X_cur_1_alum_110 + X_cur_1_alum_111 + X_cur_1_alum_112 + X_cur_1_alum_113 + X_cur_1_alum_114 + X_cur_1_alum_115 + X_cur_1_alum_116 + X_cur_1_alum_117 + X_cur_1_alum_118 + X_cur_1_alum_119 + X_cur_1_alum_12 + X_cur_1_alum_120 + X_cur_1_alum_121 + X_cur_1_alum_122 + X_cur_1_alum_123 + X_cur_1_alum_13 + X_cur_1_alum_14 + X_cur_1_alum_15 + X_cur_1_alum_16 + X_cur_1_alum_17 + X_cur_1_alum_18 + X_cur_1_alum_19 + X_cur_1_alum_2 + X_cur_1_alum_20 + X_cur_1_alum_21 + X_cur_1_alum_22 + X_cur_1_alum_23 + X_cur_1_alum_24 + X_cur_1_alum_25 + X_cur_1_alum_26 + X_cur_1_alum_27 + X_cur_1_alum_28 + X_cur_1_alum_29 + X_cur_1_alum_3 + X_cur_1_alum_30 + X_cur_1_alum_31 + X_cur_1_alum_32 + X_cur_1_alum_33 + X_cur_1_alum_34 + X_cur_1_alum_35 + X_cur_1_alum_36 + X_cur_1

##Restricciones
---

* El alumno tiene que estar en un solo curso

Para cada alumno *i*, se tiene que cumplir quela suma de su pertenencia a todos los cursos debe ser igual a 1:

$\sum_{j=1}^{n cursos}A_iC_j=1$

donde $A_iC_j$ toma el valor de 1 cuando el alumno *i* es asignado al curso *j* y 0 en los demás casos.

* Cursos con cantidades similares de alumnos

Esta restriccion es para hacer que todos los cursos tengan cantidades similares de alumnos (no son iguales porque el total de alumnos puede que no sea divisible por el tamano de los cursos). Se debe generar una restricción por cada curso *j*:

$\sum_{i=1}^{alumnos}A_iC_j\ge entero(\frac{alumnos}{n cursos}$)

* Restriccion de similar proporción de niñas en cada curso

Para cada curso *j* se debe cumplir que la proporción de niñas debe ser similar a la del total de alumnos:

$\sum_{i=1}^{alumnos}A_iC_j*Nina_{Ai}\ge entero(\frac{\sum_{i=1}^{alumnos}Nina_{Ai}}{n cursos}$)

Donde $Nina_{Ai}$ toma el valor 1 cuando la alumna *i* es niña y 0 cuando no.

* Restricciones de cantidad maxima por curso original

 Debe haber una restricción por cada par curso de origen-curso de destino.
Esta restricción es para hacer que todos los cursos tengan cantidades similares de alumnos de los cursos originales (no son iguales porque el total de alumnos puede que no sea divisible por el tamano de los cursos). Para cada curso *j* y cada curso originario *c*:

$\sum_{i=1}^{alumnos}A_iC_j*A_iC_c\ge entero(\frac{alumnos}{ncursosorig*ncursos})$

Esta restricción se puede usar para asignar de manera balanceada alumnos con distintas características.

* Restricciones de alumnos incompatibles

No deben estar en el mismo curso dos alumnos incompatibles. Esta restricción requiere que para cada alumno, exista su par incompatible. Se debe generar una restricción para cada alumno con incompatibilidad en cada curso. Para cada par de alumnos incompatibles *inc1* y *inc2* en cada curso *j*:

$A_{inc1}C_j+A_{inc2}C_j\le1$

Nótese que si más de dos alumnos son incompatibles, también es posible representarlo como pares de incompatibilidades.


* Cada alumno debe quedar con por lo menos un compañero escogido

Esta restricción se debe evaluar para todos los alumnos *i* en todos los cursos *j*.

En cada curso *j*, se debe evaluar si la suma de la participación de los escogidos $A_{esc_i,k}C_j$ menos la parcicipación del alumno suma por lo menos 0. Esta expresión solo puede ser negativa en el curso en que se asigna al alumno, por lo que para cumplirla, por lo menos uno de sus escogidos debe estar con él.

$\sum_{k=1}^{esco}A_{esc_i,k}C_j-A_iC_j\ge0$


In [90]:
# Restricciones

print('\nrest_tam_alum: cada alumno tiene que estar en un solo curso')
#Esta restriccion indica que hay y un solo alumno por curso y que todos los alumnos esten en un curso

for i in range(tot_alum):
  print( lpSum(rest_1[i][j] for j in range(num_cursos))==1)
  model+=(lpSum(rest_1[i][j] for j in range(num_cursos))==1, 'rest_un_solo_alumno '+str(i))


print('\nrest_tam_curso: cada curso tiene que tener un numero de alumnos parecido')
#Esta restriccion es para hacer que todos los cursos tengan cantidades similares de alumnos (no son iguales porque el total de alumnos puede que no sea divisible por el tamano de los cursos)

for i in range(num_cursos):
  print( lpSum(rest_1[j][i] for j in range(tot_alum))>=tam_cursos)
  model+=(lpSum(rest_1[j][i] for j in range(tot_alum))>=tam_cursos, 'rest de alumnos por curso '+str(i))

print('\nRestricciones de ninas')
# (este tipo de restricciones se puede utilizar para distribuir proporcionadamente otros grupos como los de mejor desempeno academico, etc.)
#Esta restriccion es para hacer que todos los cursos tengan cantidades similares de alumnos y alumnas (no son iguales porque el total de alumnos puede que no sea divisible por el tamano de los cursos)

for i in range(num_cursos):
  print( lpSum(rest_1[j][i]*ninas[j] for j in range(tot_alum))>=ninas_por_curso)
  model+=(lpSum(rest_1[j][i]*ninas[j] for j in range(tot_alum))>=ninas_por_curso, 'ninas por curso '+str(i))

print('\nRestricciones de cantidad maxima por curso original')
#(tiene que haber num_cursos_orig*num_cursos restricciones)
#Esta restriccion es para hacer que todos los cursos tengan cantidades similares de alumnos de distintos cursos originales (no son iguales porque el total de alumnos puede que no sea divisible por el tamano de los cursos)

for c in range(1,num_cursos_orig+1):
  for i in range(num_cursos):
    en_co=[ 1 if x ==c else 0 for x in curso_orig]
    print( lpSum(rest_1[j][i]*en_co[j] for j in range(tot_alum))>=max_mismo_curs_orig)
    model+=(lpSum(rest_1[j][i]*en_co[j] for j in range(tot_alum))>=max_mismo_curs_orig, 'curso orig '+str(c)+' en curso '+str(i))


print('\nRestricciones de alumnos incompatibles')
#Tiene que haber n_cursos restricciones por cada grupo de incompatibles
#Esta restriccion es para que no esten en el mismo cursos los alumnos incompatibles
for c in range(n_incompat):
  for i in range(num_cursos):
    incomp=[1 if x==incompat_tp[c][0] or x==incompat_tp[c][1] else 0 for x in id]
    print( lpSum(rest_1[j][i]*incomp[j] for j in range(tot_alum))<=1)
    model+=(lpSum(rest_1[j][i]*incomp[j] for j in range(tot_alum))<=1, 'incompatibles '+str(c)+' en el curso '+str(i))

print('\nRestricciones de estar por lo menos con un escogido')

for i in range(tot_alum):
  escog=[1 if x in escogidos_orig[i] else 0 for x in id]
  for k in range(num_cursos):
    print(lpSum(rest_1[j][k]*escog[j]  for j in range(tot_alum))>=rest_1[i][k])
    model+=(lpSum(rest_1[j][k]*escog[j]  for j in range(tot_alum))>=rest_1[i][k], 'escogidos alumno'+str(i)+' en el curso '+str(k))



rest_tam_alum: cada alumno tiene que estar en un solo curso
X_cur_1_alum_1 + X_cur_2_alum_1 + X_cur_3_alum_1 = 1
X_cur_1_alum_2 + X_cur_2_alum_2 + X_cur_3_alum_2 = 1
X_cur_1_alum_3 + X_cur_2_alum_3 + X_cur_3_alum_3 = 1
X_cur_1_alum_4 + X_cur_2_alum_4 + X_cur_3_alum_4 = 1
X_cur_1_alum_5 + X_cur_2_alum_5 + X_cur_3_alum_5 = 1
X_cur_1_alum_6 + X_cur_2_alum_6 + X_cur_3_alum_6 = 1
X_cur_1_alum_7 + X_cur_2_alum_7 + X_cur_3_alum_7 = 1
X_cur_1_alum_8 + X_cur_2_alum_8 + X_cur_3_alum_8 = 1
X_cur_1_alum_9 + X_cur_2_alum_9 + X_cur_3_alum_9 = 1
X_cur_1_alum_10 + X_cur_2_alum_10 + X_cur_3_alum_10 = 1
X_cur_1_alum_11 + X_cur_2_alum_11 + X_cur_3_alum_11 = 1
X_cur_1_alum_12 + X_cur_2_alum_12 + X_cur_3_alum_12 = 1
X_cur_1_alum_13 + X_cur_2_alum_13 + X_cur_3_alum_13 = 1
X_cur_1_alum_14 + X_cur_2_alum_14 + X_cur_3_alum_14 = 1
X_cur_1_alum_15 + X_cur_2_alum_15 + X_cur_3_alum_15 = 1
X_cur_1_alum_16 + X_cur_2_alum_16 + X_cur_3_alum_16 = 1
X_cur_1_alum_17 + X_cur_2_alum_17 + X_cur_3_alum_17 = 1
X_cur_1_alum_1

In [91]:
#model.solve()
model.solve(PULP_CBC_CMD())

status =  LpStatus[model.status]

print(status)

if status=='Optimal':
  print('👌')
else:
  print('☹️')


Optimal
👌


In [92]:
#print("Total Cost:", model.objective.value())


DF_result0=pd.DataFrame()

# Decision Variables

for v in model.variables():
    try:
      if v.value()>0:
        #print(v.name,"=", v.value())
        df_aux={'nombre':v.name, 'valor':v.value()}
        DF_result0=DF_result0.append(df_aux, ignore_index=True)
    except:
        print("error couldnt find value")

DF_result0['curso']=DF_result0['nombre'].str.slice( start=6, stop=7)
DF_result0['id0']=DF_result0['nombre'].str.slice( start=-4)
DF_result0[['id1', 'id']]=DF_result0['id0'].str.split('_', expand=True)
DF_result0=DF_result0.drop(columns=['id0', 'id1', 'nombre'])
DF_result0[['curso', 'id']]=DF_result0[['curso', 'id']].astype('int64')

DF_result0=DF_result0.merge(datos_entrada, on='id', how='outer')

DF_result0=DF_result0.sort_values(by=['curso', 'id'])

print(DF_result0.head())


error couldnt find value
    valor  curso  id  ...  escogido_2  escogido_3  incompatible
16    1.0      1   3  ...         109          12           NaN
0     1.0      1  10  ...          99           4           NaN
4     1.0      1  11  ...          12          71           NaN
9     1.0      1  12  ...          44          65           NaN
11    1.0      1  13  ...         123          44           NaN

[5 rows x 9 columns]


In [ ]:
print('''
Evaluacion de los resultados
____________________________

      ''')

print('\n* En cuantos cursos esta cada alumnos (duplicados)')

DF_result=DF_result0.copy()

print(DF_result['id'].value_counts().value_counts())

print(DF_result[DF_result['id'].duplicated(keep=False)])

if len(DF_result[DF_result['id'].duplicated(keep=False)])==0:
  print('👌')
else:
  print('☹️')

print('\n* Cantidad de alumnos por curso')
print(DF_result.groupby('curso')['id'].count())
if DF_result.groupby('curso')['id'].count().min()>=(tam_cursos):
  print('👌')
else:
  print('☹️')

print('\n* Cantidad de niñas por curso')
print(DF_result.groupby('curso')['nina'].sum())

if DF_result.groupby('curso')['nina'].sum().min()>=ninas_por_curso:
  print('👌')
else:
  print('☹️')

print('\n* Cantidad de curso original por curso')
print(pd.crosstab(DF_result['curso'],DF_result['curso_orig']))
if (pd.crosstab(DF_result['curso'],DF_result['curso_orig']).min().min())>=(max_mismo_curs_orig):
  print('👌')
else:
  print('☹️')


print('\n* Evaluacion de incompatibles')
eval_incomp=DF_result[~(DF_result['incompatible'].isna())][['id', 'curso', 'incompatible']]

eval_incomp['incompatible']=eval_incomp['incompatible'].astype('int64')

eval_incomp_aux=eval_incomp[['id', 'curso']]
eval_incomp_aux.columns=['incompatible', 'curso_incomp']

eval_incomp=eval_incomp.merge(eval_incomp_aux, on='incompatible', how='left')
eval_incomp['prob_incomp']=eval_incomp['curso']==eval_incomp['curso_incomp']

print(eval_incomp['prob_incomp'].value_counts())
print(eval_incomp)

if len(eval_incomp[eval_incomp['prob_incomp']])==0:
  print('👌')
else:
  print('☹️')

print('\n* Evaluacion de escogidos')

DF_escogidos=DF_result[['id', 'curso']+['escogido_'+str(x) for x in range(1,num_escogidos+1)]]
#print(DF_escogidos.head())

curso_esc=DF_result[['id', 'curso']]
DF_escogidos['eval_esc']=False

for e in range(1,num_escogidos+1):
  curso_esc.columns=['escogido_'+str(e), 'curso_e'+str(e)]
  #print(curso_esc.head())
  DF_escogidos=DF_escogidos.merge(curso_esc, on='escogido_'+str(e), how='left')
  #DF_escogidos['eval_esc']=np.where(DF_escogidos['curso']==DF_escogidos['curso_e'+str(e)], True, DF_escogidos['eval_esc'])
  DF_escogidos.loc[DF_escogidos['curso']==DF_escogidos['curso_e'+str(e)], 'eval_esc']=True

print(DF_escogidos['eval_esc'].value_counts())

if len(DF_escogidos[~DF_escogidos['eval_esc']])==0:
  print('👌')
else:
  print('☹️')


DF_result=DF_result.merge(DF_escogidos[['id', 'eval_esc']], on='id', how='left')

print('\n')

print('\n Tabla de resultados obtenidos')

DF_result


Evaluacion de los resultados
____________________________

      

* En cuantos cursos esta cada alumnos (duplicados)
1    123
Name: id, dtype: int64
Empty DataFrame
Columns: [valor, curso, id, curso_orig, nina, escogido_1, escogido_2, escogido_3, incompatible]
Index: []
👌

* Cantidad de alumnos por curso
curso
1    41
2    41
3    41
Name: id, dtype: int64
👌

* Cantidad de niñas por curso
curso
1    22
2    21
3    21
Name: nina, dtype: int64
👌

* Cantidad de curso original por curso
curso_orig   1   2   3   4
curso                     
1           10  10  11  10
2           11  10  10  10
3           10  11  10  10
👌

* Evaluacion de incompatibles
False    12
Name: prob_incomp, dtype: int64
     id  curso  incompatible  curso_incomp  prob_incomp
0    19      1           119             2        False
1    29      1           123             3        False
2    34      1            32             3        False
3    63      1            62             3        False
4    98      1   

,valor,curso,id,curso_orig,nina,escogido_1,escogido_2,escogido_3,incompatible,eval_esc
0,1.0,1,3,1,1,108,78,122,NaN,True
1,1.0,1,6,1,0,107,81,68,NaN,True
2,1.0,1,10,1,1,6,32,49,NaN,True
3,1.0,1,11,1,0,108,111,102,NaN,True
4,1.0,1,15,1,1,12,20,110,NaN,True
...,...,...,...,...,...,...,...,...,...,...
118,1.0,3,113,4,1,59,5,33,NaN,True
119,1.0,3,114,4,1,85,118,97,NaN,True
120,1.0,3,115,4,1,1,75,96,NaN,True
121,1.0,3,116,4,0,15,70,117,NaN,True
